# Tutorial 2: GPT-Adder - Learning Arithmetic with Complete Input

Welcome to the GPT-Adder tutorial! In this version, we train a transformer model to perform addition where:
- **Input (X)**: Complete question like "2+3="
- **Output (Y)**: Single predicted answer like "5"

This is different from the original autoregressive word-by-word prediction for NLP. Instead, we treat this as a sequence-to-single-token prediction task.

**Goal:** Train a transformer to map complete addition questions to single numeric answers.

- Input X is the full question "a+b="
- Output Y is a single token representing the answer
- We'll use a classification approach where each possible answer is a class
- Model architecture includes a classification head

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from transformers import GPT2Config, GPT2Model
import random
import numpy as np

# For reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

## 2. Configuration

In [2]:
# Hyperparameters
batch_size = 32     # How many independent sequences will we process in parallel to speed up the training process
max_iters = 5000       # Iteration of training 
eval_interval = 250    # Interval of evaluation
learning_rate = 1e-3   # Learning rate for the optimizer
device = 'cpu'         # The device to run the model on
eval_iters = 500       # The number of iterations to evaluate the model
n_embd = 128           # The number of embedding dimensions
n_head = 4             # The number of attention heads
n_layer = 4            # The number of layers
dropout = 0.1          # Dropout rate, this is to prevent overfitting

In [3]:
# Parameters for data generation
ndigit = 2  # Up to 2-digit numbers (0-99)

# We can view the adding problem as a classification problem, where the question is the input and the answer is the class label.
# Calculate maximum possible answer for classification
max_answer = (10**ndigit - 1) + (10**ndigit - 1)  # e.g., 99+99=198 for ndigit=2
num_answer_classes = max_answer + 1  # 0 to max_answer inclusive

print(f"Maximum possible answer: {max_answer}")
print(f"Number of answer classes: {num_answer_classes}")

Maximum possible answer: 198
Number of answer classes: 199


#### Explanation: Maximum Possible Answer and Classification

In this tutorial, we are framing the addition problem "a+b=" as a **classification task**. This means the model's goal is not to generate the sequence of digits for the answer, but to predict which *single class* the answer belongs to.

Think of it like image classification where a model predicts if an image is a "cat," "dog," or "bird." Here, our "classes" are all the possible numerical answers the addition problems can produce.

1.  **Defining the Classes:**
    Since our input numbers `a` and `b` are limited by `ndigit` (e.g., for `ndigit=2`, numbers range from 0 to 99), there's a maximum possible sum.
    - If `ndigit=2`, the largest sum is 99 + 99 = 198.
    - The smallest sum is 0 + 0 = 0.
    So, all possible answers lie in the range \[0, 198].

2.  **`num_answer_classes`:**
    Each unique integer answer in this range becomes a distinct "class" for our model.
    - `max_answer = (10**ndigit - 1) + (10**ndigit - 1)` calculates this maximum sum.
    - `num_answer_classes = max_answer + 1` determines the total number of unique classes (from 0 up to `max_answer`, inclusive). For `ndigit=2`, this is 198 + 1 = 199 classes.

3.  **Why Classification?**
    By treating this as a classification problem:
    - The model's output layer (the "classification head") will have `num_answer_classes` neurons.
    - Each neuron corresponds to one possible sum (e.g., neuron 0 for answer "0", neuron 1 for answer "1", ..., neuron 198 for answer "198").
    - The model will output a probability distribution over these classes, and the class with the highest probability is chosen as the predicted answer.
    - We use `CrossEntropyLoss`, which is standard for classification tasks.

This approach simplifies the problem compared to generating an answer character by character, especially since the output (the sum) is a single entity. The model just needs to learn to map the input question sequence to the correct answer "bucket" or class.

## 3. Data Preparation

### 3.1 Vocabulary and Tokenization

In [4]:
#We can view each character as a token, and the input is a sequence of tokens.
input_chars = '0123456789+= '  # Added space at the end for padding (will be explained later)
input_vocab_size = len(input_chars)
print(f"Input vocabulary: '{input_chars}'")
print(f"Input vocabulary size: {input_vocab_size}")

# Create mappings for input
input_stoi = {ch: i for i, ch in enumerate(input_chars)} #mapping from input elements to index
input_itos = {i: ch for i, ch in enumerate(input_chars)} #mapping from index to input elements

def encode_input(s):
    return [input_stoi[c] for c in s] #encode the input string into a list of indices

def decode_input(l):
    return ''.join([input_itos[i] for i in l]) #decode the list of indices into a string

# Test encoding/decoding
test_question = "2+3="
encoded_test = encode_input(test_question)
decoded_test = decode_input(encoded_test)
print(f"Original question: '{test_question}'")
print(f"Encoded: {encoded_test}")
print(f"Decoded: '{decoded_test}'")

Input vocabulary: '0123456789+= '
Input vocabulary size: 13
Original question: '2+3='
Encoded: [2, 10, 3, 11]
Decoded: '2+3='


### 3.2 Data Generation

In [5]:
def generate_addition_data(num_digits):
    """Generate a single addition problem and answer."""
    a = random.randint(0, 10**num_digits - 1)
    b = random.randint(0, 10**num_digits - 1)
    c = a + b
    question = f"{a}+{b}="
    answer = c  # Single integer answer
    return question, answer

# Test data generation
print("Sample problems:")
for _ in range(5):
    q, a = generate_addition_data(ndigit)
    print(f"Question: '{q}' -> Answer: {a}")

Sample problems:
Question: '81+14=' -> Answer: 95
Question: '3+94=' -> Answer: 97
Question: '35+31=' -> Answer: 66
Question: '28+17=' -> Answer: 45
Question: '94+13=' -> Answer: 107


### 3.3 Dataset Class

In [6]:
# Recall the training would be done in batches. However, the question length can not be the same for each batch.
# E.g., the question "2+3=" is 4 characters, but the question "99+99=" is 6 characters.
# So we need to pad the question to the same length for each batch.
# This is like what we do to fill the missing values in SKlearn.

# Calculate maximum question length for padding. The maximum question length is like the block_size in the nanoGPT tutorial.
max_question_length = ndigit + 1 + ndigit + 1  # a + "+" + b + "="
print(f"Maximum question length: {max_question_length}")

def get_batch():
    """
    Generate a batch of padded addition problems. 
    """
    questions = []
    answers = []
    for _ in range(batch_size):
        # Generate a new addition problem
        q_str, ans_int = generate_addition_data(ndigit)
        
        # Pad and encode the question
        # ljust(max_question_length): pad the question to the left with spaces to make it the same length as the maximum question length
        # This is like what we do to fill the missing values in the dataset.
        padded_q = q_str.ljust(max_question_length)
        encoded_q = encode_input(padded_q)
        
        questions.append(encoded_q)
        answers.append(ans_int)
    
    # Convert lists to tensors and move to the correct device
    x = torch.tensor(questions, dtype=torch.long, device=device)
    y = torch.tensor(answers, dtype=torch.long, device=device)

    return x, y 

# Sample the first question and answer in the batch
x,y = get_batch()
for i in range(5):
    print('-'*100)
    print(f"Sample question: '{decode_input(x[i].tolist())}'")
    print(f"Sample answer: '{y[i]}'")
    


Maximum question length: 6
----------------------------------------------------------------------------------------------------
Sample question: '86+94='
Sample answer: '180'
----------------------------------------------------------------------------------------------------
Sample question: '69+11='
Sample answer: '80'
----------------------------------------------------------------------------------------------------
Sample question: '75+54='
Sample answer: '129'
----------------------------------------------------------------------------------------------------
Sample question: '4+3=  '
Sample answer: '7'
----------------------------------------------------------------------------------------------------
Sample question: '11+27='
Sample answer: '38'


#### Explanation: Padding Input Sequences

Transformer models, like the GPT-2 architecture we're using as a base, are designed to process sequences of a fixed length. However, our input questions (e.g., "1+2=", "10+5=", "99+99=") can have varying lengths.

**Why Padding?**
1.  **Batch Processing:** To train neural networks efficiently, we feed data in batches. All sequences within a single batch must have the same length so they can be processed in parallel by the GPU or CPU.
2.  **Fixed-Size Model Input:** The transformer architecture itself expects inputs of a predefined maximum sequence length (`n_positions` in `GPT2Config`, which we set to `max_question_length`).

This is just like in Sklearn, when we want to use rows with missing value, we need to first fill these missing values to make every row has the same length.

**How Padding Works:**
1.  **Determine `max_question_length`:** We first calculate the maximum possible length an input question can have. For `ndigit=2`, the longest question is "99+99=" (6 characters). This becomes our `max_question_length`.
2.  **Add a Padding Token:** We add a special padding character to our input vocabulary (in this case, a space ' ').
3.  **Pad Shorter Sequences:** Any question shorter than `max_question_length` is padded with this special character (usually at the end) until it reaches the `max_question_length`.
    - "2+3=" (length 4) with `max_question_length=6` becomes "2+3=  " (length 6).
    - The `ljust(max_question_len)` method handles this.

**(Optional) Attention Mechanism and Padding:**
While the input sequences are padded, the transformer's attention mechanism can be designed (often through an attention mask) to ignore these padding tokens during computation. This ensures that the padding doesn't negatively influence the learning process. For this specific `GPT2Model` from Hugging Face, it typically handles attention masking internally based on standard padding token IDs or by allowing explicit attention masks. In our simplified setup, the model will still "see" the padding tokens, but their embeddings will be learned like any other token. The key is that the *structure* of the input is now uniform across the batch.

This padding ensures that all input tensors passed to the model have a consistent shape, which is essential for the underlying computations and batching.

## 4. Model Definition

We'll use GPT2Model (without the language modeling head) and add our own classification head.

In [7]:
class AdditionClassifier(nn.Module): #This is a class that inherits from the nn.Module class in PyTorch. 
    #Basicly it tells Python that this is a neural network model and can be used to train and test with commands in PyTorch.
    """Transformer model for addition classification."""
    
    def __init__(self, input_vocab_size, num_classes, max_seq_len, n_embd, n_layer, n_head, dropout):
        super().__init__()
        #the __init__ needs to define the architecture and parameters of the model.
        
        # GPT2 configuration for training the model from scratch
        config = GPT2Config(
            vocab_size=input_vocab_size,
            n_positions=max_seq_len,
            n_embd=n_embd,
            n_layer=n_layer,
            n_head=n_head,
            resid_pdrop=dropout,
            embd_pdrop=dropout,
            attn_pdrop=dropout,
            bos_token_id=None,
            eos_token_id=None
        )
        
        # Use GPT2Model (without language modeling/LM head) as a feature extractor. 
        #We can think this transformer/GPT2Model is a feature extractor, and its output is a feature vector of the input. 
        #These features would contain some information about the input, just like embeddings in the Word2vec/Net2vec.   
        self.transformer = GPT2Model(config)
        
        # Classification head is a linear layer that maps the feature vector to the logits of the answer.
        # Indeed we are just applying multinomial logistic regression to the feature vector from the transformer.
        self.classifier = nn.Linear(n_embd, num_classes)
     
        
    def forward(self, input_ids, labels=None):
        # The forward function is to compute the loss and the logits from the input. (Get the prediction.)


        # Get transformer outputs (the feature vector of the input)
        transformer_outputs = self.transformer(input_ids)
        feature_vector = transformer_outputs.last_hidden_state[:, -1, :]  # We get the last hidden state of the transformer as our feature vector.
 
        
        # Apply classification head/run the multinomial logistic regression (without softmax) to the feature vector from the transformer.
        logits = self.classifier(feature_vector)  
        
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss() #This is the loss function for the multinomial logistic regression, and the softmax is included in the loss function.
            loss = loss_fn(logits, labels)
        
        return {'loss': loss, 'logits': logits} #we define the output of the model as a dictionary, which contains the loss and the logits.

# Create model
model = AdditionClassifier(
    input_vocab_size=input_vocab_size,
    num_classes=num_answer_classes,
    max_seq_len=max_question_length,
    n_embd=n_embd,
    n_layer=n_layer,
    n_head=n_head,
    dropout=dropout
)

model.to(device)
print(f"{sum(p.numel() for p in model.parameters())/1e6:.2f} M parameters")

0.82 M parameters


#### Explanation: Custom Classification Head

The standard GPT-2 model from the `transformers` library, when used as `GPT2LMHeadModel`, is designed for **language modeling**. This means its primary goal is to predict the next token in a sequence, autoregressively. It has a "language modeling head" which is essentially a linear layer that maps the transformer's output hidden states to logits over the entire vocabulary (to predict the next word/character).

**Our Task is Different:**
In this tutorial, we are not performing traditional language modeling. Our task is **many-to-one classification**:
-   **Input (Many):** A sequence of characters representing an addition problem (e.g., "23+45=").
-   **Output (One):** A single class label representing the numerical answer (e.g., class 68).

**Why `GPT2Model` + Custom Head?**

1.  **Leveraging Transformer Power:** We still want to use the powerful sequence processing capabilities of the transformer architecture (self-attention, positional encodings, etc.) to understand the input question "23+45=". `GPT2Model` provides the core transformer blocks (embedding layer, multiple transformer layers) without the final language modeling layer.

2.  **Tailoring to Classification:**
    -   The output of `GPT2Model` is a sequence of hidden states, one for each input token. For our classification task, we are particularly interested in the information aggregated by the transformer over the entire sequence. A common strategy is to use the hidden state of the *last* token (or a special `[CLS]` token if one were used, or an aggregation like pooling). In our case, we use the hidden state corresponding to the final input token (which is often the '=' sign or a padding token if the actual question is shorter).
    -   This chosen hidden state (a vector of size `n_embd`) is then fed into our custom **classification head**.

3.  **The `self.classifier`:**
    Our classification head is a simple `nn.Linear` layer: `self.classifier = nn.Linear(n_embd, num_classes)`.
    -   It takes the `n_embd`-dimensional feature vector from the transformer.
    -   It projects this vector into a `num_classes`-dimensional space. Each dimension in this output corresponds to one of the possible numerical answers (from 0 to `max_answer`).
    -   The output of this linear layer are the **logits** for our classification task. Applying a softmax function to these logits gives the probabilities for each possible answer class.

In summary, we use `GPT2Model` as a powerful feature extractor for our input sequence and then add a simple linear layer (`self.classifier`) on top to perform the final classification into one of the `num_answer_classes`. This adapts the general-purpose transformer architecture to our specific arithmetic task.

## 5. Training

In [8]:
# Optimizer. We use AdamW as the optimizer like nanoGPT.
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = []
        accuracies = []
        for _ in range(eval_iters):
            # Generate a batch of data
            X, Y = get_batch()
            
            outputs = model(X, labels=Y)
            loss = outputs['loss']
            logits = outputs['logits']
            
            losses.append(loss.item())
            
            # Calculate accuracy
            predictions = torch.argmax(logits, dim=-1)
            accuracy = (predictions == Y).float().mean().item()
            accuracies.append(accuracy)
            
        out[split + '_loss'] = np.mean(losses)
        out[split + '_acc'] = np.mean(accuracies)
    model.train()
    return out

In [9]:
# Training loop
print(f"Training on {device}...")

for iter_num in range(max_iters):
    # Evaluation part
    if iter_num % eval_interval == 0 or iter_num == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter_num}: train loss {losses.get('train_loss', float('nan')):.4f}, train acc {losses.get('train_acc', float('nan')):.4f}, val loss {losses.get('val_loss', float('nan')):.4f}, val acc {losses.get('val_acc', float('nan')):.4f}")

    # Training part

    # 1. Get a batch of data
    questions, answers = get_batch()

    # 2. Forward pass
    outputs = model(questions, labels=answers)
    
    # 3. Calculate loss
    loss = outputs['loss']

    # 4 and 5. Backward pass and update parameters
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


Training on cpu...
step 0: train loss 5.5264, train acc 0.0063, val loss 5.5268, val acc 0.0069
step 250: train loss 3.8002, train acc 0.0509, val loss 3.7940, val acc 0.0493
step 500: train loss 3.4174, train acc 0.0659, val loss 3.4177, val acc 0.0642
step 750: train loss 3.1778, train acc 0.0841, val loss 3.1719, val acc 0.0823
step 1000: train loss 2.9834, train acc 0.1010, val loss 2.9765, val acc 0.1053
step 1250: train loss 2.8332, train acc 0.1088, val loss 2.8277, val acc 0.1145
step 1500: train loss 2.6516, train acc 0.1446, val loss 2.6534, val acc 0.1427
step 1750: train loss 2.4139, train acc 0.2033, val loss 2.4199, val acc 0.1989
step 2000: train loss 1.6901, train acc 0.4544, val loss 1.6861, val acc 0.4588
step 2250: train loss 1.3634, train acc 0.5609, val loss 1.3564, val acc 0.5631
step 2500: train loss 1.0856, train acc 0.6967, val loss 1.0966, val acc 0.6970
step 2750: train loss 0.7378, train acc 0.8196, val loss 0.7383, val acc 0.8177
step 3000: train loss 0.552

## 6. Testing and Evaluation

In [10]:
def test_model_addition(num_tests=20, num_digits_test=ndigit):
    print(f"--- Testing model on {num_tests} examples (up to {num_digits_test}-digit numbers) ---")
    model.eval()
    correct_predictions = 0
    with torch.no_grad():
        for i in range(num_tests):
            # Generate test problem
            a = random.randint(0, 10**num_digits_test - 1)
            b = random.randint(0, 10**num_digits_test - 1)
            correct_answer = a + b
            question = f"{a}+{b}="
            
            # Pad and encode question
            padded_question = question.ljust(max_question_length)
            encoded_question = torch.tensor(encode_input(padded_question), dtype=torch.long, device=device).unsqueeze(0) #unsqueeze(0): add a dimension at the beginning of the tensor to match the format of the training data
            
            # Get model prediction
            
            outputs = model(encoded_question)
            logits = outputs['logits']
            predicted_answer = torch.argmax(logits, dim=-1).item()
            
            is_correct = (predicted_answer == correct_answer)
            if is_correct:
                correct_predictions += 1
                status = "CORRECT"
            else:
                status = "INCORRECT"
            
            print(f"Problem {i+1:2d}: {question}{correct_answer} -> Model predicted: {predicted_answer} -> {status}")
    
    accuracy = (correct_predictions / num_tests) * 100
    print(f"Accuracy: {accuracy:.2f}% ({correct_predictions}/{num_tests} correct)")
    
    model.train()

# Run test
test_model_addition(num_tests=30, num_digits_test=ndigit)

--- Testing model on 30 examples (up to 2-digit numbers) ---
Problem  1: 69+67=136 -> Model predicted: 136 -> CORRECT
Problem  2: 87+27=114 -> Model predicted: 114 -> CORRECT
Problem  3: 29+26=55 -> Model predicted: 55 -> CORRECT
Problem  4: 10+15=25 -> Model predicted: 25 -> CORRECT
Problem  5: 97+26=123 -> Model predicted: 123 -> CORRECT
Problem  6: 22+63=85 -> Model predicted: 85 -> CORRECT
Problem  7: 21+86=107 -> Model predicted: 107 -> CORRECT
Problem  8: 5+60=65 -> Model predicted: 65 -> CORRECT
Problem  9: 22+91=113 -> Model predicted: 113 -> CORRECT
Problem 10: 76+43=119 -> Model predicted: 119 -> CORRECT
Problem 11: 23+26=49 -> Model predicted: 49 -> CORRECT
Problem 12: 1+99=100 -> Model predicted: 100 -> CORRECT
Problem 13: 89+3=92 -> Model predicted: 92 -> CORRECT
Problem 14: 93+71=164 -> Model predicted: 174 -> INCORRECT
Problem 15: 22+88=110 -> Model predicted: 110 -> CORRECT
Problem 16: 46+42=88 -> Model predicted: 88 -> CORRECT
Problem 17: 93+91=184 -> Model predicted: 

### (Optional) Interactive Testing

In [11]:
def ask_adder(problem_input):
    """Ask the model to solve an addition problem."""
    model.eval()
    
    # Ensure input ends with '='
    if not problem_input.endswith('='):
        question = problem_input + '='
    else:
        question = problem_input
    
    # Pad and encode
    padded_question = question.ljust(max_question_length)
    encoded_question = torch.tensor(encode_input(padded_question), dtype=torch.long, device=device).unsqueeze(0)
    
    # Get prediction
    with torch.no_grad():
        outputs = model(encoded_question)
        logits = outputs['logits']
        predicted_answer = torch.argmax(logits, dim=-1).item()
        
        # Also get top-3 predictions with probabilities
        probs = F.softmax(logits, dim=-1)
        top_probs, top_indices = torch.topk(probs, k=3, dim=-1)
        
    model.train()
    
    print(f"Question: {question}")
    print(f"Predicted answer: {predicted_answer}")
    print("Top 3 predictions:")
    for i in range(3):
        ans = top_indices[0][i].item()
        prob = top_probs[0][i].item()
        print(f"  {ans}: {prob:.3f}")
    
    return predicted_answer

# Test examples
print("=== Interactive Testing ===")
ask_adder('2+3')
print()
ask_adder('5+4')
print()
ask_adder('9+9')

=== Interactive Testing ===
Question: 2+3=
Predicted answer: 5
Top 3 predictions:
  5: 0.265
  4: 0.182
  6: 0.144

Question: 5+4=
Predicted answer: 11
Top 3 predictions:
  11: 0.444
  8: 0.236
  9: 0.077

Question: 9+9=
Predicted answer: 17
Top 3 predictions:
  17: 0.734
  16: 0.122
  13: 0.030


17